В этом ДЗ вам предлагается предсказать экспрессию белков по приведенным данным для отложенной выборки.

# Посмотрим на данные 

In [376]:
# библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [377]:
# Сменим рабочую дирректорию для удобства
import os
os.chdir('/home/slawa/HDD/my_scripts/my_otus/ML_advanced/lesson_23/hw')

In [378]:
# Загрузим и сохраним сырые данные

#Список ребер графа 
edges = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/edges.csv", sep=",") # Подгрузим данные
edges.to_csv('edges.csv', decimal='.', sep=',', index=False)

#Подгрузим тренирочную выборку
train = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/train.csv", sep=",") # Подгрузим данные
train.to_csv('train.csv', decimal='.', sep=',', index=False)

# Подгрузим отложенную выборку для валидации
test = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/test.csv", sep=",")
test.to_csv('test.csv', decimal='.', sep=',', index=False)

## Граф

Используем эту статью для случайного блуждания

https://lukaskoestler.com/blog/random_walk.html

In [379]:
# библиотеки
import networkx as nx
from gensim.models import Word2Vec
import random
import numpy as np
import pandas as pd

In [380]:
# функция случайного блуждания
def nx_random_walk(G, num_walks=10, num_steps=3):
    walks = list()
    for i in G.nodes():
        for walk in range(num_walks):
            curr_walk = [i]
            curr = i
            for step in range(num_steps):
                neighbors = list(G.neighbors(curr))
                if len(neighbors) > 0:
                    curr = random.choice(neighbors)
                curr_walk.append(curr)
            walks.append(curr_walk)
    return walks

In [431]:
# параметры

# длина цепочки блуждания
walk_len = 10

# количество проходов случайного блуждания 
n_walks = 5

# количество фичей в embedding
emb_size = 50

# количество исполнителей для Word2Vec
num_workers = 4

In [432]:
# Загрузим данные с описанием графа
edges = pd.read_csv("edges.csv", sep=",")
edges

,node_1,node_2
0,344,50
1,344,153
2,344,532
3,344,679
4,344,986
...,...,...
594169,14857,3544
594170,25040,11204
594171,697,20457
594172,27498,26633


In [433]:
edges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594174 entries, 0 to 594173
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   node_1  594174 non-null  int64
 1   node_2  594174 non-null  int64
dtypes: int64(2)
memory usage: 9.1 MB


In [434]:
# Может есть дубликаты?
edges.duplicated().sum()

0

In [435]:
# Создаем граф по датасету
G = nx.from_pandas_edgelist(edges, "node_1", "node_2", create_using=nx.Graph())

In [436]:
# сколько узлов получилось?
G.number_of_nodes()

10000

In [437]:
# сколько ребер получилось?
G.number_of_edges()

594174

In [438]:
# пройдем случайно по графу
walks = nx_random_walk(G, num_walks=walk_len, num_steps=n_walks)

In [439]:
# переведем случайное блуждание в вектора
str_walks = [list(map(str, w)) for w in walks]
model = Word2Vec(str_walks, vector_size=emb_size, workers=num_workers)

## Готовим датасеты

### train

In [440]:
# Загрузим выборки
train = pd.read_csv("train.csv", sep=",")
train

,target,node
0,0.251968,11142
1,0.689541,2243
2,0.678245,15514
3,0.272500,20944
4,0.248888,8721
...,...,...
7995,0.329241,19479
7996,0.350863,2182
7997,0.288309,20535
7998,1.064300,9769


In [441]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   target  8000 non-null   float64
 1   node    8000 non-null   int64  
dtypes: float64(1), int64(1)
memory usage: 125.1 KB


In [442]:
train.describe()

,target,node
count,8000.000000,8000.000000
mean,0.531021,13278.330125
std,0.673008,8750.366724
min,0.162075,0.000000
25%,0.263916,5207.750000
50%,0.339945,12825.500000
75%,0.521007,20751.250000
max,12.216818,31204.000000


In [443]:
# сборка датасета
columns = [f'x_{i}' for i in range(emb_size)]
data = []
for idx_train_df in range(train.shape[0]):
    target, node = train.iloc[idx_train_df].to_list()
    row = {'Y': target}
    row.update(dict(zip(columns, model.wv[str(int(node))])))
    data.append(row)

train_df = pd.DataFrame(data=data)
train_df

,Y,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,...,x_40,x_41,x_42,x_43,x_44,x_45,x_46,x_47,x_48,x_49
0,0.251968,-0.001020,-0.048727,0.066147,0.012178,-0.067995,-0.085497,0.171345,0.239853,-0.176358,...,0.118380,-0.125427,-0.018311,0.114259,0.253913,0.062563,-0.062092,-0.109253,0.093273,0.164929
1,0.689541,-0.019744,-0.090400,0.157287,0.047658,-0.183025,-0.293567,0.432445,0.585329,-0.417801,...,0.293336,-0.298530,-0.051692,0.262250,0.666860,0.229246,-0.179873,-0.282889,0.184459,0.400523
2,0.678245,-0.007397,-0.087965,0.125613,-0.002961,-0.236848,-0.312174,0.563352,0.674759,-0.483530,...,0.354279,-0.311993,-0.029468,0.282659,0.716932,0.236397,-0.188307,-0.334766,0.190122,0.503268
3,0.272500,-0.016002,-0.059159,0.084116,-0.000972,-0.138861,-0.180462,0.346913,0.421961,-0.310786,...,0.227581,-0.191356,-0.029098,0.189686,0.473313,0.149561,-0.130897,-0.213649,0.106165,0.293746
4,0.248888,-0.013379,-0.062290,0.109861,0.002645,-0.169161,-0.250041,0.433779,0.533454,-0.343768,...,0.247988,-0.244614,-0.037864,0.233341,0.581793,0.174117,-0.148669,-0.255567,0.138328,0.373018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0.329241,0.001186,-0.054722,0.068768,0.030594,-0.165156,-0.210049,0.349346,0.452031,-0.324358,...,0.208526,-0.229973,-0.009254,0.178823,0.499575,0.165079,-0.115904,-0.215804,0.122568,0.321271
7996,0.350863,0.013610,-0.060646,0.061159,0.031021,-0.150186,-0.200647,0.312790,0.410716,-0.272133,...,0.215450,-0.214533,-0.026794,0.166710,0.454252,0.119595,-0.137730,-0.201593,0.138203,0.303642
7997,0.288309,-0.018181,-0.081320,0.076081,0.000825,-0.165930,-0.192490,0.378021,0.494457,-0.338005,...,0.241315,-0.238879,-0.029270,0.208579,0.520377,0.179551,-0.115742,-0.205631,0.138174,0.344731
7998,1.064300,0.021293,-0.113020,0.112693,-0.001971,-0.245904,-0.278066,0.542217,0.645958,-0.461473,...,0.337910,-0.317394,-0.017933,0.265323,0.678591,0.239076,-0.151563,-0.317867,0.169280,0.470748


In [444]:
# сохранение датасета
train_df.to_csv('train_df.csv', decimal='.', sep=',', index=False)

### test

In [445]:
# Загрузим выборки
test = pd.read_csv("test.csv", sep=",")
test

,target,node
0,0.279231,817
1,0.380795,9574
2,0.686527,1607
3,0.303594,4782
4,0.367374,24125
...,...,...
1995,0.327932,1849
1996,0.278870,16977
1997,0.233343,5952
1998,0.866416,9067


In [446]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   target  2000 non-null   float64
 1   node    2000 non-null   int64  
dtypes: float64(1), int64(1)
memory usage: 31.4 KB


In [447]:
test.describe()

,target,node
count,2000.000000,2000.0000
mean,0.569324,13103.7940
std,0.867445,8726.0327
min,0.163269,8.0000
25%,0.266700,5314.7500
50%,0.341745,12341.0000
75%,0.529549,20662.5000
max,15.098584,31120.0000


In [448]:
# сборка датасета
columns = [f'x_{i}' for i in range(emb_size)]
data = []
for idx_train_df in range(test.shape[0]):
    target, node = test.iloc[idx_train_df].to_list()
    row = {'Y': target}
    row.update(dict(zip(columns, model.wv[str(int(node))])))
    data.append(row)

test_df = pd.DataFrame(data=data)
test_df

,Y,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,...,x_40,x_41,x_42,x_43,x_44,x_45,x_46,x_47,x_48,x_49
0,0.279231,-0.007718,-0.088587,0.087734,-0.020351,-0.157946,-0.213148,0.392382,0.487528,-0.318807,...,0.235470,-0.234740,-0.019300,0.185186,0.519007,0.175723,-0.130044,-0.220723,0.120079,0.333024
1,0.380795,-0.001996,-0.072500,0.067634,0.025915,-0.165763,-0.219077,0.383283,0.485264,-0.315610,...,0.255348,-0.245559,-0.036428,0.215659,0.502589,0.150045,-0.137558,-0.242822,0.159262,0.351411
2,0.686527,-0.006430,-0.094554,0.134333,0.008507,-0.215172,-0.291869,0.534811,0.686403,-0.465515,...,0.324982,-0.309399,-0.042835,0.273468,0.712177,0.233407,-0.149761,-0.324797,0.171828,0.487158
3,0.303594,0.004700,-0.081496,0.088808,0.021656,-0.177320,-0.206089,0.377110,0.486095,-0.314363,...,0.238234,-0.218307,-0.023520,0.207680,0.516526,0.145978,-0.127211,-0.215494,0.163426,0.314528
4,0.367374,0.016812,-0.053002,0.083814,-0.012830,-0.165854,-0.185099,0.344134,0.433982,-0.312586,...,0.228434,-0.182360,-0.022606,0.158765,0.473275,0.121819,-0.106700,-0.196193,0.127133,0.287153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.327932,-0.015397,-0.066626,0.068613,-0.015973,-0.142882,-0.183765,0.377307,0.445914,-0.319409,...,0.234604,-0.218114,-0.009295,0.165263,0.462956,0.151754,-0.114035,-0.214703,0.101679,0.311264
1996,0.278870,-0.009399,-0.069192,0.063828,0.018468,-0.140144,-0.184351,0.368334,0.445300,-0.316371,...,0.219550,-0.198714,-0.018329,0.181898,0.486561,0.139606,-0.126123,-0.219496,0.144481,0.318463
1997,0.233343,0.013153,-0.073989,0.084222,-0.012913,-0.144258,-0.199398,0.355040,0.452165,-0.294924,...,0.241219,-0.229359,-0.028111,0.201294,0.492944,0.162116,-0.110025,-0.211581,0.120441,0.342719
1998,0.866416,0.011463,-0.108000,0.156406,0.022609,-0.241406,-0.317339,0.494311,0.684523,-0.467316,...,0.319527,-0.313841,-0.060471,0.280144,0.739291,0.242195,-0.179706,-0.323657,0.215098,0.480970


In [449]:
# сохранение датасета
test_df.to_csv('test_df.csv', decimal='.', sep=',', index=False)

# Обучение моделей

In [450]:
# библиотеки
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
import os

In [451]:
# Сменим рабочую дирректорию для удобства
import os
os.chdir('/home/slawa/HDD/my_scripts/my_otus/ML_advanced/lesson_23/hw')

In [452]:
# загрузка датасетов
train = pd.read_csv("train_df.csv", sep=",")
test = pd.read_csv("test_df.csv", sep=",")

Y_train = train['Y'].to_numpy().reshape(-1, 1)
X_train = train.drop(['Y'], axis=1)

Y_test= test['Y'].to_numpy().reshape(-1, 1)
X_test = test.drop(['Y'], axis=1)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(8000, 50) (8000, 1)
(2000, 50) (2000, 1)


In [453]:
scaler = MinMaxScaler()
scaler.fit(Y_train)

Y_train_scale = scaler.transform(Y_train)
Y_test_scale = scaler.transform(Y_test)

In [454]:
# собираем конвейер
numeric_col = list(X_train.columns)
data_transformer = ColumnTransformer(
    [
        ("numeric_col", "passthrough", numeric_col)
    ]
)

preprocessor = Pipeline(steps=[("data_transformer", data_transformer)])

model_pipline = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('classifier', RandomForestRegressor())])
model_pipline

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('data_transformer',
                                  ColumnTransformer(transformers=[('numeric_col',
                                                                   'passthrough',
                                                                   ['x_0',
                                                                    'x_1',
                                                                    'x_2',
                                                                    'x_3',
                                                                    'x_4',
                                                                    'x_5',
                                                                    'x_6',
                                                                    'x_7',
                                                                    'x_8',
                                                                    'x_9',
                                                                    'x_10',
                                                                    'x_11',
                                                                    'x_12',
                                                                    'x_13',
                                                                    'x_14',
                                                                    'x_15',
                                                                    'x_16',
                                                                    'x_17',
                                                                    'x_18',
                                                                    'x_19',
                                                                    'x_20',
                                                                    'x_21',
                                                                    'x_22',
                                                                    'x_23',
                                                                    'x_24',
                                                                    'x_25',
                                                                    'x_26',
                                                                    'x_27',
                                                                    'x_28',
                                                                    'x_29', ...])]))])),
                ('classifier', RandomForestRegressor())])

In [455]:
# обучаем, предсказываем, проверяем
model_pipline.fit(X_train, Y_train_scale)

preds = model_pipline.predict(X_test)

print(mean_squared_error(Y_test_scale, preds))

/home/slawa/HDD/my_scripts/my_otus/ML_advanced/venv/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.000887390355819837
